<p><a href="https://www.upwork.com/jobs/Python-Sales-Data-Analysis_~01206da2beb9fe65db/?referrer_url_path=find_work_home">Upwork Link</a>

This script formats raw sales data from Toast, Square, and Clover (to be

*   List item
*   List item

implemented) into data adhering to the Flapjack data format.

**The Flapjack data format has the following columns:**

- `date`: Date of the order

- `item`: Name of the item ordered

- `price`: Price of the item

- `order_id`: Unique ID of the order this item is associated with

Each row is associated with a line item in an order. If there are multiple of the same item in a single order (i.e. two cheeseburgers in one order), there are two separate but identical rows for cheeseburgers.

In [ ]:
import pandas as pd

In [ ]:
def convert_dollars_to_float(df, columnName):
    df[columnName] = df[columnName].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
'''Expects data of the format { date, item, qty, price, order_id }'''
def standardize_quantity(df):
    df_copy = df.copy()
    df_copy['to_delete'] = False

    rows_to_append = []
    for index, row in df_copy.iterrows():
        if row['qty'] > 1:
            df_copy.at[index, 'to_delete'] = True
            for i in range(row['qty']):
                new_row = {
                    'date': row['date'],
                    'item': row['item'],
                    'price': row['price'] / row['qty'],
                    'order_id': row['order_id'],
                }
                rows_to_append.append(new_row)
    df_copy = df_copy[df_copy['to_delete'] == False]
    df_copy.drop(columns=['to_delete', 'qty'], inplace=True)

    new_df = pd.DataFrame(rows_to_append)
    df = pd.concat([df, new_df])

    return df_copy

In [ ]:
import pandas as pd

def convert_dollars_to_float(df, columnName):
  df[columnName] = df[columnName].replace('[\$,]', '', regex=True).astype(float)

'''Expects data of the format { date, item, qty, price, order_id }'''
def standardize_quantity(df):
    df_copy = df.copy()
    df_copy['to_delete'] = False

    rows_to_append = []
    for index, row in df_copy.iterrows():
        if row['qty'] > 1:
            df_copy.at[index, 'to_delete'] = True
            for i in range(row['qty']):
                new_row = {
                    'date': row['date'],
                    'item': row['item'],
                    'price': row['price'] / row['qty'],
                    'order_id': row['order_id'],
                }
                rows_to_append.append(new_row)
    df_copy = df_copy[df_copy['to_delete'] == False]
    df_copy.drop(columns=['to_delete', 'qty'], inplace=True)

    new_df = pd.DataFrame(rows_to_append)
    df = pd.concat([df, new_df])

    return df_copy

def format_square_data(csv_path):
    columns_to_include = ['Date', 'Item', 'Qty', 'Net Sales', 'Transaction ID']
    df = pd.read_csv(csv_path, usecols=columns_to_include, parse_dates=['Date'])

    renamed_columns = {
        'Date': 'date',
        'Item': 'item',
        'Qty': 'qty',
        'Net Sales': 'price',
        'Transaction ID': 'order_id',
    }
    df.rename(columns=renamed_columns, inplace=True)

    convert_dollars_to_float(df, 'price')

    # Get rid of negative quantities
    df = df[df['qty'] >= 1]

    # Turn multi-item line items into single-item
    df = standardize_quantity(df)

    return df

def format_clover_data(csv_path):
    pass

def format_toast_data(csv_path):
    columns_to_include = ['Order Date', 'Menu Item', 'Qty', 'Net Price', 'Order Id']
    df = pd.read_csv(csv_path, usecols=columns_to_include, parse_dates=['Order Date'])

    renamed_columns = {
        'Order Date': 'date',
        'Menu Item': 'item',
        'Qty': 'qty',
        'Net Price': 'price',
        'Order Id': 'order_id',
    }
    df.rename(columns=renamed_columns, inplace=True)
    df['order_id'] = df['order_id'].apply(str)

    df['qty'] = df['qty'].astype(int)

    # Get rid of fractional quantities
    df = df[df['qty'] >= 1]

    # Turn multi-item line items into single-item
    df = standardize_quantity(df)

    return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Adjust the paths to point to the correct location in your Google Drive
file_path_square = '/content/drive/My Drive/items-2023-11-01-2023-12-01.csv'
file_path_toast = '/content/drive/My Drive/ItemSelectionDetails_2023_11_14-2023_12_13.csv'


In [ ]:
import pandas as pd

# Read and display the first few rows of each file to understand their structure
square_df = pd.read_csv(file_path_square)
try:
    toast_df = pd.read_csv(file_path_toast, encoding='ISO-8859-1')
except Exception as e:
    read_error = e
else:
    read_error = None

(toast_df.head() if read_error is None else read_error), square_df.head()


In [ ]:
from datetime import datetime, time

def set_square_service_times(df, service_times):
    """
    Categorize Square data records based on user-defined service times.

    :param df: DataFrame containing Square POS data.
    :param service_times: Dictionary defining service times, e.g., {'Breakfast': ('08:00', '12:00'), 'Dinner': ('12:00', '24:00')}
    :return: DataFrame with an additional 'Service' column categorizing each record.
    """
    # Function to determine the service based on a given time
    def determine_service(record_time):
        for service, (start, end) in service_times.items():
            if start <= record_time < end:
                return service
        return "Undefined"

    # Convert 'Time' column to Python time objects for comparison
    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

    # Apply the service categorization
    df['Service'] = df['Time'].apply(determine_service)

    return df




In [ ]:
# Example usage
example_service_times = {
    'Breakfast': (time(8, 0), time(12, 0)),
    'Dinner': (time(12, 0), time(23, 59))
}

# Apply thhe Squae function to tre data with example service times
square_df_service_categorized = set_square_service_times(square_df.copy(), example_service_times)
square_df_service_categorized.head()

# **How to Use the Function:**
Define your service times in a dictionary, specifying the start and end times for each service.
Call set_square_service_times with your Square data and the service times dictionary.
Example:
```
service_times = {
    'Breakfast': (time(8, 0), time(12, 0)),
    'Lunch': (time(12, 0), time(17, 0)),
    'Dinner': (time(17, 0), time(23, 59))
}

categorized_data = set_square_service_times(square_data, service_times)
```




# **Task 2 - Convert Data to Standard Format**
Now, we'll proceed with converting both Toast POS and Square POS data to the standard Flapjack format. This format includes the following columns: date, item, price, and order_id.

In [ ]:
# Converting Toast POS Data to Standard Format

# Selecting and renaming the relevant columns for Toast data
toast_columns_mapping = {
    'Order Date': 'date',
    'Menu Item': 'item',
    'Net Price': 'price',
    'Order Id': 'order_id'
}
toast_standard_df = toast_df[toast_columns_mapping.keys()].rename(columns=toast_columns_mapping)

# Convert price from string to float and order_id to string
toast_standard_df['price'] = toast_standard_df['price'].replace('[\$,]', '', regex=True).astype(float)
toast_standard_df['order_id'] = toast_standard_df['order_id'].astype(str)

# Displaying a few rows of the standardized Toast data
toast_standard_df.head()


# **Converting Square POS Data to Standard Format**
Next, I'll convert the Square POS data to the Flapjack format. Since we've already categorized the data by service times in Task 1, we'll include that in the standardized format as well.

In [ ]:
# Converting Square POS Data to Standard Format

# Selecting and renaming the relevant columns for Square data
square_columns_mapping = {
    'Date': 'date',
    'Item': 'item',
    'Gross Sales': 'price',
    'Service': 'order_id'  # Using the 'Service' column as a placeholder for 'order_id'
}
square_standard_df = square_df_service_categorized[square_columns_mapping.keys()].rename(columns=square_columns_mapping)

# Convert price from string to float
square_standard_df['price'] = square_standard_df['price'].replace('[\$,]', '', regex=True).astype(float)

# Displaying a few rows of the standardized Square data
square_standard_df.head()


# **Task 3 - Top Selling Times/Services**
In this task, we will analyze the sales data to uncover insights into:

Gross Sales Throughout the Day (Grouped by Hour): Identifying peak sales times.
Revenue by Service: Determining which service generates the most sales.
Revenue Throughout the Week: Analyzing which day of the week records the most sales.

In [ ]:
# Converting 'date' columns to datetime for both datasets
toast_standard_df['date'] = pd.to_datetime(toast_standard_df['date'])
square_standard_df['date'] = pd.to_datetime(square_standard_df['date'])

# Extracting hour from the datetime for analysis
toast_standard_df['hour'] = toast_standard_df['date'].dt.hour
square_standard_df['hour'] = square_standard_df['date'].dt.hour

# Grouping by hour and summing up the sales for Toast data
toast_sales_by_hour = toast_standard_df.groupby('hour')['price'].sum()

# Grouping by hour and summing up the sales for Square data
square_sales_by_hour = square_standard_df.groupby('hour')['price'].sum()

toast_sales_by_hour, square_sales_by_hour


In [ ]:
# Grouping Toast data by service and summing up the sales
toast_sales_by_service = toast_standard_df.groupby(toast_df['Service'])['price'].sum()

toast_sales_by_service


In [ ]:
# Extracting day of the week from the datetime (0=Monday, 6=Sunday)
toast_standard_df['day_of_week'] = toast_standard_df['date'].dt.dayofweek
square_standard_df['day_of_week'] = square_standard_df['date'].dt.dayofweek

# Grouping by day of the week and summing up the sales for Toast data
toast_sales_by_day_of_week = toast_standard_df.groupby('day_of_week')['price'].sum()

# Grouping by day of the week and summing up the sales for Square data
square_sales_by_day_of_week = square_standard_df.groupby('day_of_week')['price'].sum()

toast_sales_by_day_of_week, square_sales_by_day_of_week


# **Task 4 - Top Selling Categories**
In this task, we'll focus on analyzing the top-selling categories by:

Ranking Categories by Sales Volume: Identifying which categories generate the most sales.
Calculating Average Sale Price per Category: Finding the average price for items in each category.
Determining the Percentage of Total Sales per Category: Assessing how much each category contributes to the overall sales.
Focusing on Dine-In Sales Data Only: We'll filter out non-dine-in sales.

In [ ]:
# Convert price from string to float
square_df['Gross Sales'] = square_df['Gross Sales'].replace('[\$,]', '', regex=True).astype(float)

# Filtering Square data for Dine-In sales only
square_dine_in_df = square_df[square_df['Dining Option'] == 'For Here']

# Grouping by category for sales volume and average sale price
category_sales_volume = square_dine_in_df.groupby('Item')['Gross Sales'].sum().sort_values(ascending=False)
category_average_price = square_dine_in_df.groupby('Item')['Gross Sales'].mean()

# Calculating the percentage of total sales per category
total_sales = square_dine_in_df['Gross Sales'].sum()
category_sales_percentage = (category_sales_volume / total_sales) * 100

category_analysis_square = pd.DataFrame({
    'Sales Volume': category_sales_volume,
    'Average Price': category_average_price,
    'Percentage of Total Sales': category_sales_percentage
})

category_analysis_square.head()  # Displaying top categories


In [ ]:
# Convert price from string to float
toast_df['Net Price'] = toast_df['Net Price'].replace('[\$,]', '', regex=True).astype(float)

# Filtering Toast data for Dine-In sales only
toast_dine_in_df = toast_df[toast_df['Dining Option'] == 'Dine In']

# Grouping by menu item for sales volume and average sale price
menu_item_sales_volume = toast_dine_in_df.groupby('Menu Item')['Net Price'].sum().sort_values(ascending=False)
menu_item_average_price = toast_dine_in_df.groupby('Menu Item')['Net Price'].mean()

# Calculating the percentage of total sales per menu item
total_sales_toast = toast_dine_in_df['Net Price'].sum()
menu_item_sales_percentage = (menu_item_sales_volume / total_sales_toast) * 100

menu_item_analysis_toast = pd.DataFrame({
    'Sales Volume': menu_item_sales_volume,
    'Average Price': menu_item_average_price,
    'Percentage of Total Sales': menu_item_sales_percentage
})

menu_item_analysis_toast.head()  # Displaying top menu items


# **Task 5 - Top Selling Categories by Service**
In this task, we'll analyze the top-selling categories grouped by service.

In [ ]:
# Grouping Toast data by service and menu item for sales volume and average sale price
service_category_sales_volume = toast_dine_in_df.groupby(['Service', 'Menu Item'])['Net Price'].sum().sort_values(ascending=False)
service_category_average_price = toast_dine_in_df.groupby(['Service', 'Menu Item'])['Net Price'].mean()

# Calculating the percentage of total sales per category within each service
service_total_sales = toast_dine_in_df.groupby('Service')['Net Price'].sum()
service_category_sales_percentage = service_category_sales_volume.div(service_total_sales, level='Service') * 100

service_category_analysis_toast = pd.DataFrame({
    'Sales Volume': service_category_sales_volume,
    'Average Price': service_category_average_price,
    'Percentage of Total Sales': service_category_sales_percentage
}).reset_index()

# Displaying top menu items by service
service_category_analysis_toast.head()


# **Task 6 - Top Selling Dishes**
For this task, we will identify the top 10 selling dishes by gross sales per category, focusing on dine-in sales data only. We'll calculate:

The Gross Sales per Dish.
The Percentage of Total Sales Each Dish Represents.
The Percentage of Category Sales Each Dish Represents.

In [ ]:
# Reload the Square POS data
square_df = pd.read_csv(file_path_square)
square_df['Gross Sales'] = square_df['Gross Sales'].replace('[\$,]', '', regex=True).astype(float)

# Filtering for dine-in sales only
square_dine_in_df = square_df[square_df['Dining Option'] == 'For Here']

# Grouping by category and item for sales volume
dish_sales_volume_square = square_dine_in_df.groupby(['Category', 'Item'])['Gross Sales'].sum().sort_values(ascending=False)

# Calculating the percentage of total sales and category sales each dish represents
total_sales_square = square_dine_in_df['Gross Sales'].sum()
category_sales_square = square_dine_in_df.groupby('Category')['Gross Sales'].sum()
dish_total_sales_percentage_square = (dish_sales_volume_square / total_sales_square) * 100
dish_category_sales_percentage_square = dish_sales_volume_square.div(category_sales_square, level='Category') * 100

# Combining the data into a single DataFrame
top_dishes_square = pd.DataFrame({
    'Gross Sales': dish_sales_volume_square,
    'Percentage of Total Sales': dish_total_sales_percentage_square,
    'Percentage of Category Sales': dish_category_sales_percentage_square
})

# Displaying the top 10 dishes
top_dishes_square.head(10)


In [ ]:
# Grouping by menu item for sales volume (without category) for Toast data
dish_sales_volume_toast = toast_dine_in_df.groupby('Menu Item')['Net Price'].sum().sort_values(ascending=False)

# Calculating the percentage of total sales each dish represents
dish_total_sales_percentage_toast = (dish_sales_volume_toast / total_sales_toast) * 100

# Combining the data into a single DataFrame
top_dishes_toast_no_category = pd.DataFrame({
    'Gross Sales': dish_sales_volume_toast,
    'Percentage of Total Sales': dish_total_sales_percentage_toast
})

# Displaying the top 10 dishes
top_dishes_toast_no_category.head(10)


# **Task 7 - Top Selling Dishes by Service**
Next, we'll group the data by service and rank the top 10 dishes by greatest sales volume, again focusing on dine-in sales only. We'll calculate:

In [ ]:
# Grouping Toast data by service and menu item for sales volume
service_dish_sales_volume_toast = toast_dine_in_df.groupby(['Service', 'Menu Item'])['Net Price'].sum().sort_values(ascending=False)

# Calculating the percentage of total sales and service sales each dish represents
service_total_sales_toast = toast_dine_in_df.groupby('Service')['Net Price'].sum()
service_dish_total_sales_percentage_toast = service_dish_sales_volume_toast.div(service_total_sales_toast, level='Service') * 100

# Combining the data into a single DataFrame
top_dishes_by_service_toast = pd.DataFrame({
    'Gross Sales': service_dish_sales_volume_toast,
    'Percentage of Total Sales': service_dish_total_sales_percentage_toast
})

# Displaying the top 10 dishes by service
top_dishes_by_service_toast.head(10)


# **Task 8 - Items Commonly Sold Together**
This task involves identifying the top 20 pairs of items that are most commonly sold together.

In [ ]:
from itertools import combinations
from collections import Counter

# Re-loading the Square POS data
square_df = pd.read_csv(file_path_square)
square_df['Gross Sales'] = square_df['Gross Sales'].replace('[\$,]', '', regex=True).astype(float)

# Filter out non-dine-in sales
square_dine_in_df = square_df[square_df['Dining Option'] == 'For Here']

# Grouping items by Transaction ID to find combinations
grouped_items = square_dine_in_df.groupby('Transaction ID')['Item'].apply(list)

# Generate all item pairs within each transaction, excluding duplicates and self-pairs
item_pairs = Counter()
for items in grouped_items:
    for item_pair in combinations(set(items), 2):
        # Sorting the pair to treat different orderings as the same (e.g., A-B and B-A)
        item_pairs[tuple(sorted(item_pair))] += 1

# Get the top 20 most common pairs
top_20_pairs = item_pairs.most_common(20)

# Convert to DataFrame for further analysis
top_20_pairs_df = pd.DataFrame(top_20_pairs, columns=['Item Pair', 'Frequency'])

# Calculate additional metrics
top_20_pairs_df['Probability of Pair Sold Together'] = top_20_pairs_df['Frequency'] / len(grouped_items)
top_20_pairs_df['Total Sales Volume'] = top_20_pairs_df['Item Pair'].apply(
    lambda x: square_dine_in_df[
        square_dine_in_df['Item'].isin(x)
    ]['Gross Sales'].sum()
)

top_20_pairs_df.head()  # Display the top 5 for brevity


In [ ]:
# Reload the Toast POS data
toast_df = pd.read_csv(file_path_toast, encoding='ISO-8859-1')
toast_df['Net Price'] = toast_df['Net Price'].replace('[\$,]', '', regex=True).astype(float)

# Filter out non-dine-in sales
toast_dine_in_df = toast_df[toast_df['Dining Option'] == 'Dine In']

# Grouping items by Order Id to find combinations
grouped_items_toast = toast_dine_in_df.groupby('Order Id')['Menu Item'].apply(list)

# Generate all item pairs within each order, excluding duplicates and self-pairs
item_pairs_toast = Counter()
for items in grouped_items_toast:
    for item_pair in combinations(set(items), 2):
        # Sorting the pair to treat different orderings as the same
        item_pairs_toast[tuple(sorted(item_pair))] += 1

# Get the top 20 most common pairs for Toast data
top_20_pairs_toast = item_pairs_toast.most_common(20)

# Convert to DataFrame for further analysis
top_20_pairs_toast_df = pd.DataFrame(top_20_pairs_toast, columns=['Item Pair', 'Frequency'])

# Calculate additional metrics for Toast data
top_20_pairs_toast_df['Probability of Pair Sold Together'] = top_20_pairs_toast_df['Frequency'] / len(grouped_items_toast)
top_20_pairs_toast_df['Total Sales Volume'] = top_20_pairs_toast_df['Item Pair'].apply(
    lambda x: toast_dine_in_df[
        toast_dine_in_df['Menu Item'].isin(x)
    ]['Net Price'].sum()
)

top_20_pairs_toast_df.head()  # Display the top 5 for brevity


# **Task 9 - Categories Commonly Sold Together**
This task involves identifying categories that are commonly sold together.

In [ ]:
# Grouping items by Transaction ID to find category combinations
grouped_categories = square_dine_in_df.groupby('Transaction ID')['Category'].apply(list)

# Generate all category pairs within each transaction, excluding duplicates and self-pairs
category_pairs = Counter()
for categories in grouped_categories:
    for category_pair in combinations(set(categories), 2):
        # Sorting the pair to treat different orderings as the same
        category_pairs[tuple(sorted(category_pair))] += 1

# Get the top 20 most common category pairs
top_category_pairs = category_pairs.most_common(20)

# Convert to DataFrame for further analysis
top_category_pairs_df = pd.DataFrame(top_category_pairs, columns=['Category Pair', 'Frequency'])

# Calculate additional metrics
top_category_pairs_df['Probability of Category Pair Sold Together'] = top_category_pairs_df['Frequency'] / len(grouped_categories)
top_category_pairs_df['Total Sales Volume'] = top_category_pairs_df['Category Pair'].apply(
    lambda x: square_dine_in_df[
        square_dine_in_df['Category'].isin(x)
    ]['Gross Sales'].sum()
)

top_category_pairs_df.head()  # Display the top 5 for brevity


In [ ]:
# Handling null or problematic values in 'Menu Group'
toast_dine_in_df['Menu Group'] = toast_dine_in_df['Menu Group'].fillna('Unknown').astype(str)
grouped_menu_groups = toast_dine_in_df.groupby('Order Id')['Menu Group'].apply(list)


# Regenerate Menu Group pairs with the corrected data
menu_group_pairs_corrected = Counter()
for menu_groups in grouped_menu_groups:
    # Ensure all menu groups are strings and filter out any 'Unknown' or empty groups
    cleaned_menu_groups = [str(group) for group in menu_groups if group and group != 'Unknown']
    for menu_group_pair in combinations(set(cleaned_menu_groups), 2):
        menu_group_pairs_corrected[tuple(sorted(menu_group_pair))] += 1

# Get the top 20 most common Menu Group pairs with corrected data
top_menu_group_pairs_corrected = menu_group_pairs_corrected.most_common(20)

# Convert to DataFrame for further analysis
top_menu_group_pairs_corrected_df = pd.DataFrame(top_menu_group_pairs_corrected, columns=['Menu Group Pair', 'Frequency'])

# Calculate additional metrics for corrected data
top_menu_group_pairs_corrected_df['Probability of Menu Group Pair Sold Together'] = top_menu_group_pairs_corrected_df['Frequency'] / len(grouped_menu_groups)
top_menu_group_pairs_corrected_df['Total Sales Volume'] = top_menu_group_pairs_corrected_df['Menu Group Pair'].apply(
    lambda x: toast_dine_in_df[
        toast_dine_in_df['Menu Group'].isin(x)
    ]['Net Price'].sum()
)

top_menu_group_pairs_corrected_df.head()  # Display the top 5 for brevity


# **Task 10 - Substitutes and Compliments**

In [ ]:
# Preparing the Square POS data for substitutes and complements analysis
# We need to ensure that the data has 'item' and 'order_id' columns
import itertools
square_analysis_df = square_dine_in_df[['Item', 'Transaction ID']].rename(columns={'Item': 'item', 'Transaction ID': 'order_id'})

# Implementing the functions from the provided script
def get_order_count_map(df):
    grouped_df = df.groupby(['item', 'order_id']).mean()
    order_count_map = grouped_df.index.get_level_values(0).value_counts().to_dict()
    return order_count_map

def get_order_id_map(df):
    unique_items = df['item'].unique()
    order_id_map = {}
    for item in unique_items:
        order_id_map[item] = set(df[df['item'] == item]['order_id'].unique())
    return order_id_map

def get_substitutes_and_complements(df):
    unique_items = df['item'].unique()
    combinations = list(itertools.combinations(unique_items, 2))
    combo_df = pd.DataFrame(combinations, columns=['item_x', 'item_y'])

    order_count_map = get_order_count_map(df)
    order_id_map = get_order_id_map(df)

    combo_df['item_x_count'] = combo_df.apply(lambda row: order_count_map[row['item_x']], axis=1)
    combo_df['item_y_count'] = combo_df.apply(lambda row: order_count_map[row['item_y']], axis=1)

    combo_df['union_count'] = combo_df.apply(lambda row: len(order_id_map[row['item_x']].union(order_id_map[row['item_y']])), axis=1)
    combo_df['intersection_count'] = combo_df.apply(lambda row: len(order_id_map[row['item_x']].intersection(order_id_map[row['item_y']])), axis=1)
    combo_df['complement_ratio'] = combo_df['intersection_count'] / combo_df['union_count']
    combo_df['min_item_count'] = combo_df.apply(lambda row: min(row['item_x_count'], row['item_y_count']), axis=1)
    combo_df['substitute_ratio'] = combo_df['intersection_count'] / combo_df['min_item_count']

    return combo_df

# Running the analysis
substitutes_and_complements_df = get_substitutes_and_complements(square_analysis_df)

# Displaying a sample of the results
substitutes_and_complements_df.head()
